In [52]:
import pandas as pd
import ast
import streamlit as st
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../data/all_pages.csv')

# コラム厳選
columns_to_keep = [
    'job_offer_id',
    'job_offer_name',
    'client',
    'job_offer_areas',
    'job_offer_min_salary',
    'job_offer_max_salary',
    'job_offer_skill_names'
]

df_filtered = df.loc[:, columns_to_keep].copy()

def to_dict_if_needed(value):
    if value is None:
        return None
    if isinstance(value, dict):
        return value
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except Exception:
            return None
    if isinstance(value, float) and pd.isna(value):
        return None
    return None


def to_list_if_needed(value):
    if value is None:
        return None
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except Exception:
            return None
    if isinstance(value, float) and pd.isna(value):
        return None
    return None

#データの整形
if 'client' in df_filtered.columns:
    df_filtered['client_dict'] = df_filtered['client'].apply(to_dict_if_needed)
    df_filtered['client_name'] = df_filtered['client_dict'].apply(lambda d: d.get('name') if isinstance(d, dict) else None)
    df_filtered['employee_count'] = df_filtered['client_dict'].apply(lambda d: d.get('employee_count') if isinstance(d, dict) else None)
    df_filtered['established'] = df_filtered['client_dict'].apply(lambda d: d.get('established_at') if isinstance(d, dict) else None)
    # df_filtered['established'] = df_filtered.to_datetime('established', unit='s')
    df_filtered['established'] = pd.to_datetime(df_filtered['established'], unit='s')
    df_filtered = df_filtered.drop(['client', 'client_dict'], axis=1, errors='ignore')

if 'job_offer_skill_names' in df_filtered.columns:
    df_filtered['job_offer_skill_names'] = df_filtered['job_offer_skill_names'].apply(to_list_if_needed)

if 'job_offer_areas' in df_filtered.columns:
    df_filtered['job_offer_areas'] = df_filtered['job_offer_skill_names'].apply(to_list_if_needed)

if 'job_offer_name' in df_filtered.columns:
    df_filtered['job_tag'] = 'その他'
    df_filtered.loc[df_filtered['job_offer_name'].str.contains('データ基盤エンジニア|データエンジニア|データベースエンジニア'), 'job_tag'] = 'データエンジニア'
    df_filtered.loc[df_filtered['job_offer_name'].str.contains('データサイエンティスト|データアナリスト'), 'job_tag'] = 'データサイエンティスト'
    df_filtered.loc[df_filtered['job_offer_name'].str.contains('AIエンジニア|機械学習エンジニア|AI開発エンジニア'), 'job_tag'] = 'AIエンジニア'

In [53]:
df_filtered.head()

,job_offer_id,job_offer_name,job_offer_areas,job_offer_min_salary,job_offer_max_salary,job_offer_skill_names,client_name,employee_count,established,job_tag
0,294449,AIエージェント開発エンジニア,"[Java, Python, Go, Node.js, AWS, Docker, Azure...",500,1200,"[Java, Python, Go, Node.js, AWS, Docker, Azure...",ブーストコンサルティング 株式会社,20,2025-03-31 15:00:00,その他
1,294052,最先端AI技術を習得！未来を創る生成AIエンジニアを募集！,"[Java, Python]",500,1040,"[Java, Python]",10X 株式会社,20,2020-01-31 15:00:00,AIエンジニア
2,294554,AIエンジニア（PM候補）,"[AWS, Azure, GCP]",800,1200,"[AWS, Azure, GCP]",株式会社 Ridge-i,84,NaT,AIエンジニア
3,294524,【AI推進室】ソフトウェアエンジニア,[],750,1200,[],株式会社 カオナビ,394,2008-04-30 15:00:00,その他
4,294243,AIエンジニア(LLMを活用したAIエージェント開発),"[Python, Azure]",700,900,"[Python, Azure]",VISITS Technologies 株式会社,60,2014-05-31 15:00:00,AIエンジニア


In [56]:
df_filtered['job_tag']

0          その他
1      AIエンジニア
2      AIエンジニア
3          その他
4      AIエンジニア
        ...   
434        その他
435        その他
436        その他
437        その他
438    AIエンジニア
Name: job_tag, Length: 439, dtype: object

In [61]:
df_filtered[df_filtered['job_tag']=='データエンジニア']

,job_offer_id,job_offer_name,job_offer_areas,job_offer_min_salary,job_offer_max_salary,job_offer_skill_names,client_name,employee_count,established,job_tag
6,294081,データエンジニア（メンバーポジション）,"[SQL, Python, AWS, Azure, GCP]",400,550,"[SQL, Python, AWS, Azure, GCP]",株式会社 cococo,10,2023-11-30 15:00:00,データエンジニア
11,187679,（職種未経験者歓迎）データエンジニア,"[C, SQL, Java, Python, R, AWS, MySQL, PostgreS...",400,600,"[C, SQL, Java, Python, R, AWS, MySQL, PostgreS...",株式会社 メンバーズ メンバーズデータアドベンチャーカンパニー,2303,1995-05-31 15:00:00,データエンジニア
20,273394,データエンジニア リーダー候補,"[SQL, Java, Python, Scala, AWS, MySQL, Postgre...",600,1000,"[SQL, Java, Python, Scala, AWS, MySQL, Postgre...",株式会社 FLINTERS,90,2013-12-31 15:00:00,データエンジニア
21,286281,データエンジニア（データ基盤構築）／リードエンジニア,"[SQL, AWS, Azure, Google Cloud Platform, GCP]",500,650,"[SQL, AWS, Azure, Google Cloud Platform, GCP]",株式会社 エスタイル,63,2006-06-30 15:00:00,データエンジニア
23,267012,データエンジニア,"[C, C++, JavaScript, SQL, Java, Python, Node.j...",600,900,"[C, C++, JavaScript, SQL, Java, Python, Node.j...",株式会社 ヘッドウォータース,127,2005-10-31 15:00:00,データエンジニア
...,...,...,...,...,...,...,...,...,...,...
403,272445,データエンジニア,[],480,800,[],株式会社 識学,227,2015-02-28 15:00:00,データエンジニア
406,211742,データエンジニア（MX統括部）,"[JavaScript, SQL, Python, AWS, PostgreSQL, Azu...",500,1000,"[JavaScript, SQL, Python, AWS, PostgreSQL, Azu...",パーソルビジネスプロセスデザイン 株式会社,18253,1977-08-31 15:00:00,データエンジニア
411,229877,データ基盤エンジニア,"[JavaScript, Java, Python, AWS, MySQL, Postgre...",700,1000,"[JavaScript, Java, Python, AWS, MySQL, Postgre...",アソビュー 株式会社,200,2011-02-28 15:00:00,データエンジニア
421,207594,データベースエンジニア,[SQL],400,600,[SQL],フレックシステムズ 株式会社,56,2003-09-30 15:00:00,データエンジニア


In [ ]:
df['job_offer_areas'] = df['job_offer_areas'].apply(ast.literal_eval)
type(df.loc[2, "job_offer_skill_names"])

str

In [ ]:
import pandas as pd

# エポック秒のSeries（10桁または13桁の数値）
epoch_series = pd.Series([1401548400, 1580482800, 1209567600])

# pd.to_datetime()で変換
# unit='s' は、入力が秒であることを示します
datetime_series = pd.to_datetime(epoch_series, unit='s')

print(datetime_series)

0   2014-05-31 15:00:00
1   2020-01-31 15:00:00
2   2008-04-30 15:00:00
dtype: datetime64[ns]


In [ ]:
import pandas as pd
import ast

df = pd.read_csv('../data/filtered.csv')

df_copy = df.copy()

df_copy['job_tag'] = 'その他'  # まず'その他'で初期化

df_copy.loc[df_copy['job_offer_name'].str.contains('データ基盤エンジニア|データエンジニア|データベースエンジニア'), 'job_tag'] = 'データエンジニア'

df_copy.loc[df_copy['job_offer_name'].str.contains('データサイエンティスト|データアナリスト'), 'job_tag'] = 'データサイエンティスト'

df_copy.loc[df_copy['job_offer_name'].str.contains('AIエンジニア|機械学習エンジニア|AI開発エンジニア'), 'job_tag'] = 'AIエンジニア'


df_copy.head(50)

,job_offer_id,job_offer_name,job_offer_areas,job_offer_min_salary,job_offer_max_salary,job_offer_skill_names,client_name,employee_count,established,job_tag
0,294449,AIエージェント開発エンジニア,"['Java', 'Python', 'Go', 'Node.js', 'AWS', 'Do...",500,1200,"['Java', 'Python', 'Go', 'Node.js', 'AWS', 'Do...",ブーストコンサルティング 株式会社,20,1.743433e+09,その他
1,294052,最先端AI技術を習得！未来を創る生成AIエンジニアを募集！,"['Java', 'Python']",500,1040,"['Java', 'Python']",10X 株式会社,20,1.580483e+09,AIエンジニア
2,294554,AIエンジニア（PM候補）,"['AWS', 'Azure', 'GCP']",800,1200,"['AWS', 'Azure', 'GCP']",株式会社 Ridge-i,84,NaN,AIエンジニア
3,294524,【AI推進室】ソフトウェアエンジニア,[],750,1200,[],株式会社 カオナビ,394,1.209568e+09,その他
4,294243,AIエンジニア(LLMを活用したAIエージェント開発),"['Python', 'Azure']",700,900,"['Python', 'Azure']",VISITS Technologies 株式会社,60,1.401548e+09,AIエンジニア
5,288361,◢◤ビッグデータ人材◢◤データオペレーションスタッフ,"['SQL', 'Python']",400,800,"['SQL', 'Python']",TRUSTART 株式会社,31,1.588259e+09,その他
6,294081,データエンジニア（メンバーポジション）,"['SQL', 'Python', 'AWS', 'Azure', 'GCP']",400,550,"['SQL', 'Python', 'AWS', 'Azure', 'GCP']",株式会社 cococo,10,1.701356e+09,データエンジニア
7,294517,データサイエンティスト＜福岡銀行＞,"['SQL', 'Python', 'R']",600,1500,"['SQL', 'Python', 'R']",株式会社 福岡銀行,3718,-7.838532e+08,データサイエンティスト
8,294439,シニアデータサイエンティスト,"['SQL', 'R', 'pandas']",1000,0,"['SQL', 'R', 'pandas']",OLTA 株式会社,118,1.490972e+09,データサイエンティスト
9,294323,ソフトウェアエンジニア（AI）,"['Ruby', 'Ruby on Rails']",600,1600,"['Ruby', 'Ruby on Rails']",株式会社 kickflow,34,1.580483e+09,その他
